<p style="color:#FFF; background:#06D; padding:12px; font-size:20px; font-style:italic; text-align:center">
<span style="width:49%; display:inline-block; text-align:left">Version 2025-06</span>
<span style="width:49%; display:inline-block; text-align:right">Licence CC–BY–NC–ND</span>
<span style="font-size:40px; font-style:normal"><b>INTERPRÉTEUR : IPYTHON</b></span><br>
<span style="width:49%; display:inline-block; text-align:left">Christophe Schlick</span>
<span style="width:49%; display:inline-block; text-align:right">schlick ಄ u<b>-</b>bordeaux • fr</p>

Le chapitre 3 a permis de faire un tour d'horizon de la syntaxe du langage Python, ainsi que des fonctionnalités de base fournies par le noyau de ce langage. Les éléments vus dans le chapitre 3 sont donc valables quelque soit l'environnement de développement utilisé pour écrire du code Python :
  [**IDLE**](https://docs.python.org/3/library/idle.html) &#9679;
  [**Spyder**](https://www.spyder-ide.org) &#9679;
  [**PyCharm**](https://www.jetbrains.com/pycharm) &#9679;
  [**VS Code**](https://code.visualstudio.com) &#9679; et autres...

Dans ce chapitre, nous allons aborder quelques fonctionnalités plus avancées qui sont spécifiques à l'environnement Jupyter et à l'interpréteur [**IPython**](https://ipython.org) qui s'exécute en arrière-plan sur le serveur, pour interpréter le code Python dans les cellules de code d'un notebook. Le noyau d'exécution de l'interpréteur IPython s'appelle **ipykernel** (c'est ce nom qui apparait en haut à droite dans l'interface JupyterLab pour indiquer le noyau en cours d'exécution dans le notebook actif). Comme on l'a vu au chapitre 1, c'est le seul noyau disponible par défaut à l'installation de la distribution Anaconda. Le noyau **ipykernel** a été spécifiquement conçu pour l'écosystème ***Scientific Python***, et offre ainsi des outils particulièrement intéressants pour toute personne devant développer des applications scientifiques, et notamment des applications en Sciences des Données. Ce chapitre se propose donc de faire un tour d'horizon rapide des fonctionnalités les plus utiles. Pour avoir des informations complémentaires, on peut se référer à la documentation complète du package qui se trouve sur le site [**readthedocs**](https://ipython.readthedocs.io/en/stable/), sachant qu'une copie locale est directement disponible dans le menu **`Help`** de JupyterLab, sous le titre ***IPython Reference***.

In [1]:
import warnings; warnings.filterwarnings('ignore') # suppression des 'warning' de l'interpréteur
from SRC.tools import show, inspect, cutcut # import des fonctions utilitaires du module 'tools'

<h2 style="padding:16px; color:#FFF; background:#06D">A - Debugging, benchmarking, robustesse et fiabilité</h2>

Dans le domaine du développement logiciel, indépendamment du langage utilisé et du domaine d'application concerné, il existe un consensus assez large sur les cinq critères principaux permettant d'évaluer la **qualité d'un code source** :

* La **lisibilité** du code
* La **robustesse** du code
* La **fiabilité** du code
* La **vitesse d'exécution** du code
* La **sobriété mémoire** du code

La lisibilité du code est évidemment un critère relativement subjectif, même s'il existe des règles assez précises sur les bonnes pratiques qui permettent d'améliorer cette lisibilité. Par contre, les quatre autres critères peuvent être mesurés de manière objective. L'environnement Jupyter fournit un certain nombre d'outils pour améliorer la productivité durant l'écriture du code puis, lorsque le code est opérationnel, de mesurer facilement les éléments liés à la vitesse d'exécution, la sobriété mémoire, la robustesse et la fiabilité du code.

Pour illustrer la mise en oeuvre de ce processus, on va utiliser l'exemple très classique de la **[suite de Fibonacci](https://fr.wikipedia.org/wiki/Suite_de_Fibonacci)**, définie par la mathématicien italien [**Leonardo Fibonacci**](https://fr.wikipedia.org/wiki/Leonardo_Fibonacci) en 1202 par la relation de récurrence suivante :

$$
F_0 = 0\qquad F_1 = 1\qquad F_p = F_{p-1} + F_{p-2}\quad \forall\, p > 1
$$

En 1834, le mathématicien français [**Jacques Binet**](https://fr.wikipedia.org/wiki/Jacques_Philippe_Marie_Binet) a calculé l'expression fonctionnelle de la suite, c'est-à-dire la formule permettant d'exprimer directement le $n$-ième terme de la suite, sans avoir besoin de connaître les termes précédents :

$$
F_n = \frac{\varphi^n-\psi^n}{\varphi-\psi}\qquad \text{avec}\qquad \varphi = \frac{1+\sqrt5}2\qquad \text{et}\qquad \psi = \frac{1-\sqrt5}2
$$

A partir de ces éléments, on peut implémenter trois versions différentes pour effectuer le calcul des termes de cette suite :

- une **version récursive** **`fiboR(n)`** dans laquelle les termes sont calculés par des appels récursifs
- une **version itérative** **`fiboI(n)`** dans laquelle les termes sont calculés à l'aide d'une boucle
- une **version fonctionnelle** **`fiboF(n)`** dans laquelle les termes sont obtenus via la formule de Binet

---
### 1 - Debugging

Comme la plupart des environnements modernes utilisés pour le développement logiciel, JupyterLab intègre un [**débogueur**](https://fr.wikipedia.org/wiki/D%C3%A9bogueur), terme français officiel même si la plupart des développeurs francophones continuent d'employer le terme anglais : ***debugger***. Concrètement il s'agit d'un outil informatique permettant à un développeur d'identifier et corriger un dysfonctionnement de son code, traditionnellement appelé ***bug***, en mettant en oeuvre un **processus de recherche de panne** (***troubleshooting***) similaire à ce qui existe dans le domaine du génie industriel. Quelque soit l'environnement d'exécution utilisé, un debugger permet toujours de rajouter au mininum les quatre fonctionnalités suivantes :

- La création de **points d'arrêt** à des endroits spécifiques durant l'exécution du code
- Une exécution **pas à pas** (plus exactement, instruction par instruction) à partir d'un point d'arrêt
- L'affichage du **contenu des variables** locales et globales durant l'exécution pas à pas
- L'affichage du **contenu de la pile d'appel** des fonctions durant l'exécution pas à pas

Evidemment, l'utilisation d'un debugger est un processus foncièrement interactif, il n'est donc pas facile d'illustrer son fonctionnement dans un notebook statique comme ce chapitre de cours, mais en mettant des indications sur les actions à réaliser au clavier et à la souris, cela vous permet de reproduire les opérations décrites, sur votre session JupyterLab

La fonction **`fiboF(n)`** ci-dessous implémente la version fonctionnelle de Fibonacci en utilisant la formule de Binet. Le code ne présente aucune difficulté puisqu'il consiste simplement à définir les deux variables **`phi`** et **`psi`**, puis à effectuer le calcul indiqué par la formule. Pour éviter les approximations inévitables lors des calculs en nombres réels, il ne faut pas oublier d'arrondir le résultat vers l'entier le plus proche, puisque les termes de la suite sont nécessairement des entiers.

Pour activer le debugger dans le notebook courant, il faut commencer par ***cliquer sur l'icône ressemblant à un insecte*** noir (comme **bug** en anglais) sur la partie droite de la barre d'outils qui se trouve au-dessus de ce notebook. Normalement, après un léger temps de mise en place, l'icône doit devenir rouge pour indiquer que le debugger est actif. Le deuxième effet visuel est que toutes les cellules de code se voient rajouter une numéroration des lignes, et que la barre de contrôle à gauche de la fenêtre bascule automatiquement sur l'onglet **`Debugger`**.

La seconde étape consiste à placer un point d'arrêt à l'intérieur de la fonction, ici on va choisir de mettre cet arrêt sur la ligne 5 de la cellule de code. Pour cela, il faut cliquer sur la zone grisée, juste à gauche du numéro 5 pour faire apparaître un disque rouge indiquant la position du point d'arrêt. Ensuite, en exécutant la cellule par **<kbd>CTRL</kbd>**+**<kbd>ENTER</kbd>**, l'interpréteur commence exécuter la boucle **`for`**, mais lorsqu'il effectue le premier appel **`fibo(0)`**, l'exécution va s'arrêter à l'endroit du point d'arrêt qui vient d'être placé : dans la cellule de code, la ligne 5 est mise sur fond gris, et dans la barre de contrôle à gauche, s'affichent les valeurs stockées dans les trois variables locales **`n`**, **`phi`** et **`psi`** au moment du point d'arrêt.

In [2]:
def fiboF(n):
  """return the n-th term of the Fibonacci sequence (functional version, based on Binet's formula)
     fibo(n) = (phi**n-psi**n)/(phi-psi) where phi = (1+sqrt(5))/2 and psi = (1-sqrt(5))/2"""
  phi, psi = (1 + 5**0.5) / 2, (1 - 5**0.5) / 2
  return round((phi**n - psi**n) / (phi - psi)) # round result to nearest integer

for n in range(10): print(f"F{n} = {fiboF(n)}", end=' / ') # display the 10 first Fibonacci terms

F0 = 0 / F1 = 1 / F2 = 1 / F3 = 2 / F4 = 3 / F5 = 5 / F6 = 8 / F7 = 13 / F8 = 21 / F9 = 34 / 

Lorsqu'on se trouve sur un point d'arrêt, plusieurs options sont possibles pour l'utilisateur, en utilisant les différents boutons de la barre d'exécution se trouvant en-dessous de la zone d'affichage des variables. Les rôles des 6 boutons sont récapitulés sur la figure ci-dessous, mais pour cette première fonction, on va simplement cliquer sur le bouton <tt>▶</tt> (= **continue**) ce qui va continuer l'exécution jusqu'au prochain point d'arrêt. Ce nouvel arrêt correspond au deuxième appel de la fonction **`fiboF`**, avec cette fois-ci **`n=1`**. En cliquant sur le boutton <tt>▶</tt> à chaque point d'arrêt, cela permet d'exécuter le code étape par étape, en s'arrêtant à chaque nouvel appel de fonction, et on constate que l'affichage des termes de la suite s'effectue un par un, en dessous de la cellule de code. A l'inverse, appuyer sur le bouton <tt>■</tt> (= **terminate**) de la barre d'exécution, ne provoque pas un arrêt de l'exécution (comme on pourrait s'y attendre) mais un arrêt du debugger, et par conséquent toutes les itérations restantes de la boucle **`for`** vont s'exécuter jusqu'au bout, sans interruption.

<center><img src="IMG/debugger.png" width=256></center>

---
On voit que la fonction **`fiboF(n)`** implique le calcul de trois constantes **`phi`**, **`psi`** et **`1/(phi-psi)`** dont l'évaluation fait intervenir des racines carrées et des divisions, deux opérations assez coûteuses pour le CPU. Une optimisation classique, très facile à mettre en place, consiste à précalculer l'ensemble des constantes utilisées par une fonction. Dans notre cas, lorsqu'on évalue un seul terme de la suite, cela ne va pas engendrer une différence notable pour le temps d'exécution, mais si l'on doit calculer plusieurs milliers de termes, la différence est loin d'être négligeable, comme on le verra dans la section suivante. On va donc écrire une seconde version de la fonction, appelée **`fiboP(n)`**, en précalculant les trois constantes :
$$
\varphi = 1.618033988749895\qquad \psi = -0.6180339887498949\qquad \tau = \frac{1}{\varphi - \psi} = 0.4472135954999579
$$

In [3]:
def fiboP(n):
  """return the n-th term of the Fibonacci sequence (precomputed functional version)
     fibo(n) = (phi**n-psi**n)/(phi-psi) where phi = (1+sqrt(5))/2 and psi = (1-sqrt(5))/2"""
  return round((1.618033988749895**n - (-0.6180339887498949)**n) * 0.4472135954999579)

for n in range(10): print(f"F{n} = {fiboP(n)}", end=' / ') # display the 10 first Fibonacci terms

F0 = 0 / F1 = 1 / F2 = 1 / F3 = 2 / F4 = 3 / F5 = 5 / F6 = 8 / F7 = 13 / F8 = 21 / F9 = 34 / 

---
La troisième version **`fiboR(n)`** va utiliser la définition par récurrence et produire une fonction récursive pour le calcul des termes de la suite. Là encore, la fonction est d'une simplicité extrême puisqu'elle consiste uniquement à traduire en Python, la définition mathématique, en utilisant l'opérateur d'évaluation conditionnelle. On peut même simplifier encore l'expression en fusionnant les deux cas $F_0$ et $F_1$, comme le montre le code ci-dessous.

La définition de la suite de Fibonacci est une récurrence à deux termes (il faut connaître $F_{n-2}$ et $F_{n-1}$ pour calculer $F_n$), ce qui se traduit en Python par deux appels récursifs dans le code de la fonction **`fiboR`**. Autrement dit, chaque appel de fonction va générer deux appels de fonction, qui vont générer quatre appels et ainsi de suite. Pour mieux comprendre le fonctionnement de la récursion, 

In [4]:
def fiboR(n):
  """return the n-th term of the Fibonacci sequence (recursive version)"""
  return fiboR(n-1) + fiboR(n-2) if n > 1 else n # merge n=0 and n=1 in a single test

for n in range(10): print(f"F{n} = {fiboR(n)}", end=' / ') # display the 10 first Fibonacci terms

F0 = 0 / F1 = 1 / F2 = 1 / F3 = 2 / F4 = 3 / F5 = 5 / F6 = 8 / F7 = 13 / F8 = 21 / F9 = 34 / 

In [5]:
def fiboM(n, memo={0:0,1:1}):
  """return the n-th term of the Fibonacci sequence (recursive version with memoization)"""
  if n in memo: return memo[n] # return value already stored in 'memo' dictionary
  else: memo[n] = fiboM(n-1, memo) + fiboM(n-2, memo); return memo[n] # add new value in dictionary

for n in range(10): print(f"F{n} = {fiboM(n)}", end=' / ') # display the 10 first Fibonacci terms

F0 = 0 / F1 = 1 / F2 = 1 / F3 = 2 / F4 = 3 / F5 = 5 / F6 = 8 / F7 = 13 / F8 = 21 / F9 = 34 / 

In [6]:
def fiboT(n, a=0, b=1):
  """return the n-th term of the Fibonacci sequence (terminal recursive version)"""
  return fiboT(n-1, b, a+b) if n > 1 else b if n > 0 else a

for n in range(10): print(f"F{n} = {fiboT(n)}", end=' / ') # display the 10 first Fibonacci terms

F0 = 0 / F1 = 1 / F2 = 1 / F3 = 2 / F4 = 3 / F5 = 5 / F6 = 8 / F7 = 13 / F8 = 21 / F9 = 34 / 

In [7]:
def fiboI(n):
  """return the n-th term of the Fibonacci sequence (iterative version)"""
  a, b = 0, 1 # initialisation des variables stockant les termes de la suite : a = F_0, b = F_1
  for _ in range(n): # utilisation de la variable anonyme '_' comme compteur de boucle
    a, b = b, a + b # mise à jour des termes de la suite : 'a' = terme précédent, 'b' = terme courant
  return a

for n in range(10): print(f"F{n} = {fiboI(n)}", end=' / ') # display the 10 first Fibonacci terms

F0 = 0 / F1 = 1 / F2 = 1 / F3 = 2 / F4 = 3 / F5 = 5 / F6 = 8 / F7 = 13 / F8 = 21 / F9 = 34 / 

In [8]:
def fiboG(n):
  """return the n-th term of the Fibonacci sequence (generator version)"""
  a, b = 0, 1 # initialisation des variables stockant les termes de la suite : a = F_0, b = F_1
  for _ in range(n-1): # utilisation de la variable anonyme '_' comme compteur de boucle
    yield a; a, b = b, a + b # mise à jour des termes de la suite : 'a' = terme précédent, 'b' = terme courant
  yield a

for n,f in enumerate(fiboG(10)): print(f"F{n} = {f}", end=' / ') # display the 10 first Fibonacci terms

F0 = 0 / F1 = 1 / F2 = 1 / F3 = 2 / F4 = 3 / F5 = 5 / F6 = 8 / F7 = 13 / F8 = 21 / F9 = 34 / 

In [9]:
for c in 'FPMTI': # loop over fibo versions
  name = 'fibo' + c; f = eval(name) # get name and code for current fibo version
  a, b = f(100), str(f(1000)) # compute the 100th and the 1000th term for current fibo version
  print(f"● {name}(100) = {a} ● {name}(1000) = {b[:20]}...{b[-20:]}") # cut middle of fibo(1000)

● fiboF(100) = 354224848179263111168 ● fiboF(1000) = 43466557686938914862...34523370463746326528
● fiboP(100) = 354224848179263111168 ● fiboP(1000) = 43466557686938914862...34523370463746326528
● fiboM(100) = 354224848179261915075 ● fiboM(1000) = 43466557686937456435...76137795166849228875
● fiboT(100) = 354224848179261915075 ● fiboT(1000) = 43466557686937456435...76137795166849228875
● fiboI(100) = 354224848179261915075 ● fiboI(1000) = 43466557686937456435...76137795166849228875


---
### 2 - Benchmarking

Lorsqu'on travaille en traitements des données, il arrive très fréquemment que l'on se retrouve dans une situation similaire à celle de notre exemple, c'est-à-dire avoir plusieurs implémentations alternatives pour une même fonctionnalité. Mis à part certains cas triviaux, sélectionner la meilleure version est rarement une tâche facile, car il n'y a pas forcément une relation d'ordre total, selon les critères de comparaison choisis. L'opération de comparaison des performances de ces différentes implémentations s'appelle un **benchmark**, qui à l'origine était un terme utilisé par les géomètres et les cartographes pour identifier une altitude de référence sur le terrain (la traduction littérale signifie ***marque sur un banc de mesure***)

Un critère de comparaison important pour ces différentes implémentations consiste à mesurer leur vitesse d'exécution sur un même exemple (on choisira généralement un exemple plutôt complexe, pour lequel les temps de calcul risquent d'être assez élevés). Le noyau **`ipykernel`** permet d'effectuer de telles comparaisons très facilement grâce à la commande magique **`%timeit`** qui va créer un ***environnement de benchmarking*** pour la mesure de performance entre plusieurs algorithmes.

Dans notre exemple de la suite de Fibonacci, comme on dispose de cinq versions différentes pour effectuer le même calcul, il est tout à fait judicieux de lancer un benchmarking pour obtenir en faisant plusieurs tests comparatifs sur les vitesses de calcul. Dans un premier test, on compare le temps d'exécution pour le calcul des 25 premiers termes de la suite de Fibonacci :

In [10]:
a = %timeit fiboI(25)

966 ns ± 10.2 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


In [11]:
for c in 'FPRMTI':
  name = 'fibo' + c; f = eval(name); print('● ' + name)
  %timeit [f(n) for n in range(25)]
%timeit list(fiboG(25))

● fiboF
10.5 μs ± 147 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
● fiboP
11.2 μs ± 704 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
● fiboR
22.8 ms ± 912 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)
● fiboM
2.31 μs ± 203 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
● fiboT
20.8 μs ± 1.58 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
● fiboI
10.8 μs ± 1.06 μs per loop (mean ± std. dev. of 7 runs, 100,000 loops each)
1.62 μs ± 33.3 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


On constate que pour de petites valeurs de **$n$** :
  - **`fiboF`**, **`fiboP`** et **`fiboI`** ont des temps d'exécution très similaires
  - **`fiboR`** est environ 2000x plus lente que **`fiboI`**
  - **`fiboT`** est environ 2x plus lente que **`fiboI`**
  - **`fiboL`** est environ 5x plus rapide que  **`fiboI`**

Lorsqu'on augmente la valeur de **$n$**, les temps de calcul avec **`fiboR`** deviennent très vite rédhibitoires (plusieurs minutes pour **$n = 50$** ), à cause des calculs inutiles engendrés par la double récurrence. Par conséquent, on ne va conserver que les quatre autres versions pour le second test qui consiste à calculer les 1000 premiers termes de la suite de Fibonacci :

In [12]:
for c in 'MTI':
  name = 'fibo' + c; f = eval(name); print('● ' + name)
  %timeit [f(n) for n in range(1000)]

● fiboM
99.2 μs ± 2.6 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
● fiboT
76.7 ms ± 4.59 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
● fiboI
20.2 ms ± 210 μs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [13]:
for c in 'MTI':
  name = 'fibo' + c; f = eval(name); print('● ' + name)
  %timeit [f(n) for n in range(1000)]

● fiboM
96.2 μs ± 1.79 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)
● fiboT
77.6 ms ± 4.48 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)
● fiboI
20.3 ms ± 1.25 ms per loop (mean ± std. dev. of 7 runs, 100 loops each)


On peut constater que, pour des valeurs de *n* plus grandes, la version itérative **`fibo`** est près de 50x plus lente que la version fonctionnelle **`fiboF`** . Donc, avec ce seul critère de la **vitesse d'exécution**, on pourrait conclure que **`fiboF`** est une implémentation largement meilleure que **`fibo`**.

Mais la vitesse d'exécution n'est pas le seul critère qualitatif pour juger de l'implémentation d'un algorithme. Un autre critère fondamental est le **domaine de validité** qui définit l'ensemble des données pour lesquelles l'algorithme donne un résultat correct. Et sur ce critère, la différence entre les deux versions est sans appel :

La version itérative effectue ses calculs en utilisant le type **`int`** pour lequel (comme on l'a rappelé dans le chapitre précédent) la précision n'est limitée que la quantité de mémoire disponible sur l'interpréteur Python. Dans notre cas, on obtient sans aucune difficulté, les 2090 chiffres composant la valeur exacte du 10000$^e$ terme de la suite de Fibonacci :

In [14]:
fiboI(9999)

2079360823713349807211264898864283682508703609401590311968294586652850142345568664892745603430522651559175734329719015801062479426725097317613381017990273803823178974834623555648319143159192453239442002806781032040872441469346284906266838708330804825092065449334087873322637758084744632487379760373479464825811385863155040408101726038120291994389237094285260164739821355447908182359371542956694514931299366484677909043779928477367537928427066017513466483326637769864201210689135579114187277693408080350495679409464829288056605636471818766266897075853738335267742083557415594565854200363476532454100612101244678568917149480326240860269309121160197393822944663604990153196328615969907788042772028923553932967187718291564341907918652511867885682160089752017107049943765706734240087108390881180097625972743182053955425686946081535591845825339823438236043576275982317989611674842426954592463320461413799285081435201873848092358155398899089715146940613169561449778372074346137375621868510685682609069633981

A l'inverse, la version fonctionnelle effectue ses calculs en utilisant le type **`float`** pour lequel (comme on l'a également rappelé dans le chapitre précédent) le stockage est défini avec une précision limitée (*norme IEEE 754*). Dans notre cas, la valeur du 10000$^e$ terme dépasse très largement le plus grand nombre représentable par le type **`float`** et donc l'exécution de la fonction aboutit à un erreur de débordement **`OverflowError`**

In [15]:
# Note : enlever le caractère '#' en préfixe de la ligne suivante pour provoquer l'erreur
#fiboM(9999)

Même s'il est simplifié à l'extrême, cet exemple est assez caractéristique d'une situation qu'on retrouve souvent en Analyse Numérique, et plus généralement dans les Sciences des Données : pour un même traitement, il existe des algorithmes rapides mais avec un domaine de validité restreint et des algorithmes plus lents avec des domaines plus vastes. Pour un problème donné, il faut donc avoir une connaissance en amont assez précise sur la variabilité des données à traiter, afin de choisir l'algorithme optimal pour chaque cas de figure.

In [ ]:
from numba import jit # import de la fonction 'jit' (just-in-time compilation)

@jit
def fiboNF(n):
  """return the n-th term of the Fibonacci sequence (numba optimized functional version)
     fibo(n) = (phi**n-psi**n)/(phi-psi) where phi = (1+sqrt(5))/2 and psi = (1-sqrt(5))/2"""
  phi, psi = (1 + 5**0.5) / 2, (1 - 5**0.5) / 2
  return round((phi**n - psi**n) / (phi - psi))

@jit
def fiboNP(n):
  """return the n-th term of the Fibonacci sequence (numba optimized precomputed functional version)
     fibo(n) = (phi**n-psi**n)/(phi-psi) where phi = (1+sqrt(5))/2 and psi = (1-sqrt(5))/2"""
  return round((1.618033988749895**n - (-0.6180339887498949)**n) * 0.4472135954999579)

@jit
def fiboNR(n):
  """return the n-th term of the Fibonacci sequence (numba optimized recursive version)"""
  return fiboNR(n-1) + fiboNR(n-2) if n > 1 else n

@jit
def fiboNT(n, a=0, b=1):
  """return the n-th term of the Fibonacci sequence (terminal recursive version)"""
  return fiboNT(n-1, b, a+b) if n > 1 else b if n > 0 else a

@jit
def fiboNI(n):
  """return the n-th term of the Fibonacci sequence (numba optimized iterative version)"""
  a, b = 0, 1
  for _ in range(n): a, b = b, a + b
  return a

In [ ]:
n = 100
for f in (fiboF, fiboP, fiboM, fiboT, fiboNT, fiboI, fiboNI): print(f(n) % 1000000)

---
Mais on peut faire encore bien mieux que cela en termes de performance. En effet, le noyau d'exécution **ipykernel** inclut par défaut les fonctionnalités du package [**numba**](https://numba.pydata.org) qui propose un mécanisme de [**compilation à la volée**](https://fr.wikipedia.org/wiki/Compilation_%C3%A0_la_vol%C3%A9e) (***Just-In-Time*** ou **JIT**) qui permet de convertir une fonction Python en langage machine une 

In [ ]:
%timeit -r 20 [fiboNF(n) for n in range(1000)]
%timeit -r 20 [fiboNP(n) for n in range(1000)]
%timeit -r 20 [fiboNT(n) for n in range(1000)]
%timeit -r 20 [fiboNI(n) for n in range(1000)]

In [ ]:
%timeit fiboNI(30000)
#%timeit fiboNT(20000)
#%timeit fiboI(20000)

In [ ]:
%timeit fiboNP(30000)

In [ ]:
fiboI(3000) % 1000000, fiboNI(3000) % 1000000, fiboNP(3000) % 1000000

---
### 2 - Robustesse et contrôle des données entrantes

La robustesse d'un programme se définit comme sa capacité à détecter en amont les données problématiques, avant qu'elles ne provoquent des erreurs sur les traitements à mettre en oeuvre.

La manière la plus simple pour augmenter la robustesse d'un programme consiste à mettre ***un contrôle de données entrantes*** au début du bloc de chacune des fonctions du programme qui manipule des données provenant d'une source extérieure (par exemple, des données qui ont été soit saisies au clavier par un utilisateur, soit lues dans un fichier local ou distant, soit récupérés via un flux réseau). Ce type de contrôle est tellement fréquent (et tellement important) que la plupart des langages de programmation fournissent une instruction spécifique pour cette tâche.

Ainsi le langage Python possède une instruction **`assert <test>, <message>`** qui va interrompre le programme en affichant le message **`<message>`** à chaque fois que **`<test>`** fournit une valeur booléenne fausse. Il suffit donc pour une donnée particulière d'identifier toutes les conditions que doit respecter cette donnée, et de les combiner avec l'opérateur **`and`** pour construire l'expression **`<test>`**.

> **Note :** Dans l'instruction **`assert`**, la virgule de séparation entre le test et le message d'erreur est obligatoire. De plus, il arrive assez fréquemment (notamment si le test combine plusieurs conditions qui doivent être vérifiées simultanément) que le nombre de caractères nécessaire pour l'ensemble de la commande **`assert`** dépasse largement les 80 caractères qui représente la taille maximale conseillée pour une ligne de code en Python. La solution habituelle dans ces cas-là est d'insérer un caractère de continuation **`'\'`** juste après la virgule, et de placer le message d'erreur sur la ligne suivante. Pour les mêmes raisons, si le test comporte un grand nombre de conditions à vérifier, il est généralement préférable d'utiliser plusieurs instructions **`assert`** avec des tests et des messages d'erreurs spécifiques.

Voici ce que cela donne pour la fonction **`fibo(n)`** :

In [ ]:
# on rajoute un contrôle de type et de valeur sur la donnée entrante 'n' de la fonction
def fibo(n):
  """return the n-th term of the Fibonacci sequence (iterative version)"""
  assert isinstance(n, int) and n >= 0, f"{n} must be a positive integer"
  a, b = 0, 1
  for loop in range(n):
    a, b = b, a + b
  return a

In [ ]:
# Note : enlever un par un le caractère '#' en préfixe des lignes suivantes pour tester
#fibo(-1)
#fibo(1.5)
#fibo('toto')

---
### 3 - Fiabilité et tests unitaires

La fiabilité d'un programme se définit comme sa capacité à fournir systématiquement un résultat correct, lorsque le jeu de données fourni en entrée se trouve dans le domaine de validité pour le traitement souhaité. Comme pour la robustesse, la fiabilité d'un programme se vérifie systématiquement au niveau de chacune des fonctions qui le composent. Le principe général mis en oeuvre pour le contrôle de la fiabilité, consiste à écrire, quasiment en même temps que le code de chacune des fonctions, un ensemble de cas d'utilisation, appelée ***tests unitaires*** (*unit test*, en anglais) qui sont destinés à vérifier sur un jeu de données suffisament représentatif, que la fonction fournit bien les résultats attendus.

Le langage Python possède un package **`pytest`** qui fournit un moteur de tests unitaires similaires à ceux qui existent dans d'autres langages de programmation. Son principe consiste à écrire un ensemble de fonctions (toutes préfixées par **`test_...`**) qui contiennent un ensemble de clauses **`assert`** fournissant des conditions sur les valeurs de retours pour différents appels de fonctions. Le moteur va ainsi identifier et exécuter automatiquement toutes les fonctions de tests (grâce au préfixe) est vérifier que toutes les conditions sont validées. Le package **`pytest`** est assez simple à mettre en oeuvre, mais il est conçu pour du développement logiciel basé sur un IDE classique, où le code est composé d'un ensemble de modules. Ainsi pour chaque module **`xxx`** qui compose son code, le développeur va écrire un autre module **`test_xxx`** qui sera dédié aux tests des fonctions du module **`xxx`**.

Lorsqu'on développe en Python à l'intérieur d'un notebook, le code n'est pas décomposé en modules indépendants, mais en une série de cellules de code. Par conséquent, le package **`pytest`** n'est pas très facile à mettre en oeuvre, et il s'avère que le package **`doctest`** est infiniment mieux adapté. Avec ce package, au lieu de créer des fonctions de tests spécifiques, le développeur va simplement inclure dans la docstring de chaque fonction, les différents cas d'utilisation de la fonction et le résultat attendu pour chaque cas. Avec une syntaxe très légère (cf. ci-dessous), cette idée permet non seulement d'inclure facilement des tests unitaires, mais également de profiter de la présence de ces tests pour enrichir la documentation de chacune des fonctions du notebook, et fournir ainsi à l'utilisateur des informations très précises sur l'utilisation de ces fonctions.

La référence complète pour l'utilisation du package **`doctest`** se trouve dans le **[manuel de référence](https://docs.python.org/fr/3/library/doctest.html#)** de la bibliothèque standard de Python, mais comme son principe est simplissime, il suffit d'un exemple illustrant les trois cas de figures usuels pour comprendre la syntaxe à respecter pour sa mise en oeuvre :

In [ ]:
from doctest import testmod # import de la fonction 'testmod' du package 'doctest'

In [ ]:
def fibo(n: int) -> int:
  """return the n-th term of the Fibonacci sequence (iterative version)
  >>> fibo(0), fibo(1), fibo(2), fibo(3), fibo(4), fibo(5)
  (0, 1, 1, 2, 3, 5)
  >>> [fibo(n) for n in range(50)] # doctest: +ELLIPSIS
  [0, 1, ... 7778742049]
  >>> fibo(-1), fibo(2.5), fibo('z')
  Traceback (most recent call last):
  ...
  AssertionError: -1 must be a positive integer
  """
  assert isinstance(n, int) and n >= 0, f"{n} must be a positive integer"
  a, b = 0, 1
  for loop in range(n):
    a, b = b, a + b
  return a

Comme on peut le voir dans le code ci-dessus, on rajoute une série de lignes simulant des appels à la fonction avec différents jeux de paramètres, dans la chaîne de caractères multi-lignes correspondant à la docstring de la fonction **`fibo`**. Chaque ligne commençant par un préfixe **`>>>`** indique les appels à effectuer, et la ou les lignes suivantes (jusqu'au prochain préfixe **`>>>`**) représentent le résultat attendu. Pour compacter l'affichage, on peut tout à fait combiner plusieurs appels en une seule ligne, en les séparant par des virgules. Par exemple:
```
>>> fibo(0), fibo(1), fibo(2), fibo(3), fibo(4), fibo(5)
(0, 1, 1, 2, 3, 5)
```

Si l'on sait que le résultat renvoyé par un appel particulier est très long (en nombre de caractères ou en nombre de lignes), on peut utiliser la notation **`...`** (en activant le mode *ELLIPSIS* par un commentaire spécifique) et se contenter de ne mettre que le début et la fin du résultat attendu :
```
>>> [fibo(n) for n in range(50)] # doctest: +ELLIPSIS
[0, 1, ... 7778742049]
```

Enfin, si l'on sait que le résultat va renvoyer une erreur (**`AssertionError`**, **`TypeError`**, **`ValueError`** ou autres), il suffit là encore de ne mettre que le début et la fin du message affiché, pour que **`doctest`** puisse vérifier que le résultat obtenu est bien conforme au résultat attendu. Même si concrètement seul le premier appel de fonction provoquant une erreur est exécuté, il est assez fréquent de combiner tous les cas d'utilisation provoquant la même erreur sur la même ligne, pour gagner de la place dans la docstring :
```
>>> fibo(-1), fibo(2.5), fibo('z')
Traceback (most recent call last):
...
AssertionError: -1 must be a positive integer
```



L'appel de la fonction **`testmod`** va analyser toutes les docstrings de toutes les fonctions d'un notebook, identifier les tests qui y figurent et exécuter l'un après l'autre chacun des tests pour contrôler le résultat obtenu. Si tout se passe bien, l'affichage sera minimaliste, en indiquant simplement le nombre de test effectués et le nombre d'échecs (cf. ci-dessous). Si un ou plusieurs tests échouent, l'affichage sera plus détaillé, permettant au développeur d'identifier toutes les erreurs rencontrées. On peut également activer l'option **`verbose=True`** pour forcer la fonction **`testmod`** à détailler l'ensemble des tests effectués :

In [ ]:
testmod()
#testmod(verbose=True) # enlever le '#' pour obtenir la version détaillée

<h2 style="padding:16px; color:#FFF; background:#06D">B - Affichage de données multimedia</h2>

Comme on l'a vu dans le chapitre précédent, le noyau du langage Python (et c'est le cas pour la quasi-totalité des noyaux des langages de programmation généralistes) n'inclut que les outils de base pour ***manipuler des données numériques et des données textuelles***. Lorsqu'on souhaite manipuler des données plus complexes (graphiques, photos, audios, vidéos, bases de données structurées, ...) il faudra impérativement utiliser des bibliothèques d'extension, ne serait-ce que pour lire et afficher les données correspondantes.

Dans ce contexte, l'avantage de l'environnement JupyterLab est indéniable : comme il s'exécute dans un navigateur web, les notebooks ont directement accès à toutes les fonctionnalités permettant l'affichage des données multimédia qu'on trouve habituellement dans les pages HTML. Le noyau [**ipykernel**](https://ipython.org) fournit ainsi un package **`IPython`** contenant un module **`display`** qui permet aux programmes Python de lire et d'afficher très facilement des images, des vidéos, des sons, qu'ils soient stockés dans des fichiers locaux, ou sur des serveurs distants.

---

Pour importer le module **`IPython.display`** on va utiliser la technique appelée **short alias import** qui est devenue très classique au sein de l'écosystème *Scientific Python* dans lequel les programmes sont généralement grands consommateurs de modules ou de packages importés. Comme son nom l'indique, la technique consiste à ***importer un module ou package en lui donnant un alias court*** afin de raccourcir le préfixe qu'il faudra utiliser pour accéder aux ressources qu'il contient : 

> **`import IPython.display as dp`**

Après cette commande d'import, toutes les fonctions disponibles dans le module **`IPython.display`** seront accessibles dans le notebook, simplement en préfixant le nom de la fonction par l'alias du module. Dans notre cas, une fonction **`xxx(...)`** appartenant au module pourra être appelée avec **`dp.xxx(...)`** alors qu'avec un import classique, il faudrait utiliser la préfixe complet **`IPython.display.xxx(...)`** ce qui sera moins efficace mais également moins lisible.

In [ ]:
import IPython.display as dp # import du package 'Ipython.display' avec alias 'dp'

Les cellules de code ci-dessous vont illustrer l'utilisation de quelques unes des fonctions fournies par ce module, pour visualiser des données multimédia complexes directement en Python, sans être obligé de passer par les commandes magiques comme on l'avait fait au chapitre 1 :

---
### 1 - Données au format Markdown

In [ ]:
# On crée une chaîne de caractères multi-lignes contenant un bloc de code Markdown
# Attention : il est recommandé d'utiliser systématiquement une chaîne brute
# (préfixe 'r' avant les guillemets) pour éviter que le noyau n'interprète les
# caractères qui ont un rôle en Python, notamment les backslashes '\'
code = r"""
Voici une liste regroupant différents éléments pouvant être visualisés en Markdown :
* Elément contenant des caractères Unicode par copier/coller : 🙂👍🌞⛔
* Elément contenant des caractères Unicode par code numérique : &#9824; &#9827; &#9829; &#9830;
* Elément contenant une équation mathématique : $e^{\,i\pi} + 1 = 0$
* Elément contenant un extrait de code source : `x, y = minmax(x, y)`
* Elément contenant un lien hypertexte local : [**Jupyter Logo**](IMG/jupyter.png)
* Elément contenant un lien hypertexte distant : [**Jupyter Website**](https://www.jupyter.org)
"""
dp.Markdown(code) # on affiche le code interprété via la fonction 'dp.Markdown'

In [ ]:
# Cela fonctionne également si le code se trouve dans un fichier local ou distant
dp.Markdown(filename='TEST/display.md') # mettre url=... pour visualiser un fichier distant

---
### 2 - Données au format LaTeX

In [ ]:
# On crée une chaîne de caractères multi-lignes (toujours une chaîne brute) contenant du code LaTeX
code = r"""
$$\left\{\begin{align}
e^{\,i\pi} + 1 &= 0 &&
{\sf (identité d'Euler)}\\
F(\omega) &= \int_{-\infty}^{+\infty} f(t)\,e^{-2\pi i\, \omega t}\, dt &&
{\sf (transformée de Fourier continue)}\\
S_m &= \sum_{n\,=\,0}^{N-1} s_n\, e^{-2\pi i\frac{m\,n}{N}} &&
{\sf (transformée de Fourier discrète)}
\end{align}\right.$$
"""
dp.Latex(code) # on affiche le code interprété via la fonction 'dp.Latex'

In [ ]:
# Cela fonctionne également si le code se trouve dans un fichier local ou distant
dp.Latex(filename='TEST/display.tex') # mettre url=... pour visualiser un fichier distant

---
### 3 - Données au format HTML + CSS

In [ ]:
# On crée une chaîne de caractères multi-lignes contenant du code HTML et CSS
text = r"""
<style>
  .row {display:flex; font-size:16px; font-weight:700}
  .colA {flex:66%; background:#7FA; padding:8px}
  .colB {flex:33%; background:#FA7; padding:8px}
  .colC {flex:99%; text-align:center; font-style:italic}
</style>
<div class=row>
  <div class=colA>ABCDEFGHIJKLMNOPQRSTUVWXYZ</div>
  <div class=colB>ABCDEFGHIJKLMNOPQRSTUVWXYZ</div>
</div> <div class=row>
  <div class=colC>ABCDEFGHIJKLMNOPQRSTUVWXYZ</div>
</div> <div class=row>
  <div class=colC>ABCDEFGHIJKLMNOPQRSTUVWXYZ</div>
  <div class=colC>ABCDEFGHIJKLMNOPQRSTUVWXYZ</div>
</div> <div class=row>
  <div class=colB>ABCDEFGHIJKLMNOPQRSTUVWXYZ</div>
  <div class=colA>ABCDEFGHIJKLMNOPQRSTUVWXYZ</div>
</div>
"""
dp.HTML(text) # on affiche le code interprété via la fonction 'dp.HTML'

In [ ]:
# Cela fonctionne également si le code se trouve dans un fichier local ou distant
dp.HTML(filename='TEST/display.html') # mettre url=... pour visualiser un fichier distant

---
### 4 - Données au format SVG

In [ ]:
# On crée une chaîne de caractères multi-lignes contenant du code SVG
code = r"""
<svg viewBox='0 0 750 250'>
  <g fill='#EEE' stroke='#000' stroke-width='2'>
    <rect x='25' y='25' width='700' height='200' rx='25' stroke-width='4'/>
    <ellipse cx='100' cy='125' rx='50' ry='75' fill='#F00'/>
    <ellipse cx='210' cy='125' rx='50' ry='75' fill='#0F0'/>
    <ellipse cx='320' cy='125' rx='50' ry='75' fill='#00F'/>
    <ellipse cx='430' cy='125' rx='50' ry='75' fill='#0FF'/>
    <ellipse cx='540' cy='125' rx='50' ry='75' fill='#F0F'/>
    <ellipse cx='650' cy='125' rx='50' ry='75' fill='#FF0'/>
  </g>
</svg>
"""
dp.SVG(code) # on affiche le code interprété via la fonction 'dp.SVG'

In [ ]:
# Cela fonctionne également si le code se trouve dans un fichier local ou distant
dp.SVG(filename='TEST/display.svg') # mettre url=... pour visualiser un fichier distant

---
### 5 - Données de types Image / Audio / Video

Les données de types Image, Audio ou Video sont généralement stockées dans des fichiers de données, en utilisant des formats standardisés :
- **BMP** (non compressé), **JPG**, **PNG** ou **GIF** pour les ***images***
- **WAV** (non compressé), **MP3**, **AAC** ou **OGG** pour les ***sons***
- **MP4** pour les ***vidéos***

Le package **`display`** fournit trois fonctions **`Image`**, **`Audio`** et **`Video`** permettant de visualiser les contenus de ces fichiers. Néanmoins, le comportement de ces fonctions diffèrent selon qu'on utilise un fichier local (appel avec l'argument **`filename=...`**) ou distant (appel avec l'argument **`url=...`**). Dans le premier cas, les données du fichier visualisé sont directement incluses dans le fichier **`.ipynb`** du notebook (ce qui permet de conserver l'info, même si le fichier initial est effacé ou déplacé, mais crée le risque d'un notebook devenant ***très volumineux*** si de nombreux fichiers multimédia sont insérés de cette manière. Dans le second cas, seul le lien URL est stocké dans le notebook (ce qui n'augmente donc pas la taille du fichier  **`.ipynb`** mais impose à l'utilisateur d'être connecté à Internet lors de l'utilisation du notebook). Selon les cas de figures, l'un ou l'autre de ces deux comportements peut s'avérer préférable.

In [ ]:
dp.Image('IMG/smiley.png') # mettre url=... pour visualiser un fichier distant

In [ ]:
dp.Audio('IMG/beep.mp3') # mettre url=... pour visualiser un fichier distant

In [ ]:
dp.Video('IMG/noise.mp4') # mettre url=... pour visualiser un fichier distant

Pour les vidéos en streaming, le module **`display`** propose deux fonctions **`YouTubeVideo`** et **`VimeoVideo`** permettant d'afficher une vidéo dont on connait l'identificateur alphanumérique sur les deux sites classiques [**YouTube**](https://www.youtube.com) et [**Vimeo**](https://vimeo.com/fr/) :

In [ ]:
dp.YouTubeVideo('A5YyoCKxEOU', width='100%', height=360) # affichage vidéo YouTube via son identificateur

In [ ]:
dp.VimeoVideo('455065259', width='100%', height=360) # affichage vidéo Vimeo via son identificateur

---
### 6 - Affichage de sites web

Enfin, le module **`display`** propose une fonction **`IFrame`** qui a le même rôle que la balise **`<iframe>`** en HTML, et qui sera généralement utilisée pour afficher un site web complet au sein d'un notebook :

In [ ]:
dp.IFrame('https://www.wikipedia.org', width='100%', height=360)

---
### 7 - Utilisation de 'display' dans une fonction

Lorsque la dernière expression d'une cellule de code, consiste à appeler une des fonctions du module **`display`** (comme on l'a fait dans les exemples précédents), le résultat renvoyé par cette fonction est directement interprété par le noyau d'exécution, ce qui provoque l'affichage de la donnée correspondante (Markdown, HTML, LaTeX, Image, etc). Par contre, lorsqu'une fonction de ce module est appelé à l'intérieur d'une autre fonction Python, l'affichage n'est pas automatique. Dans ce cas de figure, il faut ***explicitement appeler la fonction **`dp.display`** pour provoquer l'affichage***.

Voici un exemple de mise en oeuvre, avec le code d'une fonction **`fold_seq`** permettant d'afficher le contenu d'une séquence arbitraire en Python à l'aide d'une table HTML. Le paramètre **`fold`** définit le nombre d'éléments à mettre sur une même ligne, avant d'effectuer un changement de ligne (*line folding*, en anglais) :

In [ ]:
def fold_seq(seq, fold=10): # version sans les balises fermantes
  """display a Python sequence as an HTML table, where 'fold' controls line folding"""
  seq = [str(item) for item in seq] # convert all items to string
  seq = [seq[n:n+fold] for n in range(0, len(seq), fold)] # extract slices of 'fold' items
  head, tail = '<center><table>', '</table></center>' # set head and tail of HTML code
  rowsep, colsep = '\n<tr>', '\n  <td>' # set row and col separators as HTML tags with indentation
  html = rowsep + rowsep.join(colsep + colsep.join(row) for row in seq) # set body of HTML code
  html = '\n'.join([head, html, tail]) # insert body between head and tail
  #print(html) # print whole generated HTML code (TL;DR)
  #print(cutcut(html)) # print only head and tail lines of HTML code (use 'cutcut' from 'tools' module)
  dp.display(dp.HTML(html)) # display generated HTML table

fold_seq(range(1, 121), fold=20) # valeurs de 1 à 120, réparties par lignes de 20 colonnes

**Note :** La fonction précédente ne définit que les balises ouvrantes (**`<tr>`** et **`<td>`**) des lignes et des colonnes de la table, alors que la norme HTML impose de rajouter les balises fermantes associées (**`</tr>`** et **`</td>`**). En pratique, il s'avère que la totalité des navigateurs web savent interpréter correctement le formatage d'une table HTML, même si les balises fermantes des lignes et des colonnes sont manquantes (c'est un héritage de l'époque où la norme était moins stricte) donc la fonction **`fold_seq`** donne un résultat correct quelque soit le navigateur utilisé pour exécuter l'environnement Jupyter. Ceci dit, si on est puriste et qu'on cherche à produire du code HTML respectant la norme, on peut réécrire la fonction précédente, pour obtenir une version légèrement plus compliquée :

In [ ]:
def fold_seq(seq, fold=10): # version avec les balises fermantes
  """display a Python sequence as an HTML table, where 'fold' controls line folding"""
  seq = [str(item) for item in seq] # convert all items to string
  seq = [seq[n:n+fold] for n in range(0, len(seq), fold)] # extract slices of 'fold' items
  head, tail = '<center><table>', '</table></center>' # set head and tail of HTML code
  # define 6 separators as HTML tags : (head, tail, sep = head+tail) both for row and col
  rowhead, rowtail, colhead, coltail = '<tr>\n', '</tr> ', '  <td>', '</td>\n'
  rowsep, colsep = rowtail + rowhead, coltail + colhead
  html = rowhead + rowsep.join(colhead + colsep.join(row) + coltail for row in seq) + rowtail
  html = '\n'.join([head, html, tail]) # insert HTML body between head and tail
  #print(cutcut(html)) # print head and tail lines of HTML code
  dp.display(dp.HTML(html)) # display generated HTML table

fold_seq(range(1, 121), fold=20) # valeurs de 1 à 120, réparties par lignes de 20 colonnes

Générer du code HTML par programme avant d'utiliser la fonction **`display`** pour l'afficher, est une pratique très courante à l'intérieur des notebooks Jupyter, en particulier lorsque l'on souhaite afficher des données avec une mise en page précise. Une autre pratique courante assez similaire consiste à générer du code SVG par programme, afin de pouvoir construire une image de manière algorithmique. Voici un exemple permettant de générer un échiquier dont la taille et les couleurs peuvent être modifiées par l'utilisateur :

In [ ]:
def chessboard(size=8, scale=32, colors='#EEE #777'):
  """display a chessboard as an SVG image, using provided size, scale factor and colors"""
  head, tail = f"<svg width='{scale*size}' viewBox='0 0 {size} {size}'>", '</svg>' # set <svg> tag
  colors = colors.split() # convert colors string to a couple of colors
  svg = [f"<rect x='{c}' y='{r}' width='1' height='1' fill='{colors[(r+c)%2]}'/>"
    for r in range(size) for c in range(size)] # create list of <rect> tags for all cells
  svg = '\n'.join([head, *svg, tail]) # merge SVG body between head and tail
  #print(cutcut(svg)) # print head and tail lines of HTML code
  #dp.display(dp.SVG(svg)) # display SVG image
  dp.display(dp.HTML(f"<center>{dp.SVG(svg).data}</center>")) # display centered SVG image

chessboard() # échiquier défini avec les arguments par défaut
#chessboard(40, 12, '#0F0 #00F') # modification de la taille et des couleurs

<h2 style="padding:16px; color:#FFF; background:#06D">C - Interaction</h2>

Le noyau [**ipykernel**](https://ipython.org) fournit un autre package appelé **`ipywidgets`** qui permet d'intégrer dans les notebooks Jupyter, un ***ensemble de widgets d'interaction*** (on parle également d'**interacteurs**) pour modifier les arguments des fonctions contenues dans un notebook, sans être obligé de modifier les cellules de code correspondantes. Cela permet donc à un utilisateur qui ne maîtrise pas la programmation, de pouvoir modifier facilement les jeux de données à expérimenter.

Le package **`ipywidgets`** recèle de nombreuses fonctionnalités permettant de créer des notebooks interactifs très complexes. Pour une utilsation avancée, il faut se référer à la documentation complète du package qui se trouve sur le site [**ReadTheDocs**](https://ipywidgets.readthedocs.io). Dans cette section, nous allons simplement illustrer l'apport le plus important que fournit le package, à savoir la définition d'une fonction **`interact`** extrêmement flexible, qui permet de créer automatiquement des widgets graphiques pour la saisie interactive de données par l'utilisateur.

---
Lorsqu'on souhaite utiliser l'intégralité des fonctionnalités fournies, on importe habituellement le package **`ipywidgets`** par le biais d'un alias court, avec la commande suivante :

> **`import ipywidgets as ws`**

Lorsqu'on souhaite simplement bénéficier de la fonction **`interact`** (ce qui est notre cas), on peut directement importer cette fonction, sans avoir à utiliser le reste du package :

> **`from ipywidgets import interact`**


In [ ]:
from ipywidgets import interact # import de la fonction 'interact' du package 'ipywidgets'

In [ ]:
def test(a, b): return a, b # simple fonction de test pour les différents intéracteurs

In [ ]:
test(10, 15)

---
### 1 - Interacteurs à valeurs numériques

Les interacteurs à valeurs numériques permettent à l'utilisateur de saisir soit une valeur entière, soit une valeur réelle par le biais d'un **curseur graphique**. L'intervalle de variation de ce curseur est fournie sous la forme d'un couple d'entiers ou de réels. Une troisième valeur (optionnelle) permet de spécifier le pas d'incrément du curseur. Si ce pas n'est pas spécifié, il est initialisé à 1 pour les entiers, et à 0.1 pour les réels

In [ ]:
interact(test, a=(0,10), b=(0,50,5)); # interacteur à valeurs entières

In [ ]:
interact(test, a=(0.0,1.0), b=(-5,5,0.5)); # interacteur à valeurs réelles

---
### 2 - Interacteurs à choix

Les interacteurs à choix permettent à l'utilisateur de saisir soit une valeur booléenne (sous la forme d'une **case à cocher**), soit une chaîne de caractères (sous la forme d'une **zone de saisie** alphanumérique), soit de choisir parmi une liste de valeurs prédéfinies (sous la forme d'une **liste déroulante**). Comme pour
les interacteurs à valeurs numériques, c'est toujours par le biais de la fonction `interact` que ces interacteurs sont créés. Pour chaque argument fourni à la fonction `interact`, c'est la nature de la valeur d'initialisation de cet argument qui permet à **ipywidgets** de choisir l'interacteur adéquat

In [ ]:
interact(test, a=True, b=False); # interacteur à valeurs booléennes

In [ ]:
interact(test, a='', b=['AAA','BBB','CCC','DDD','EEE','FFF']); # interacteurs de saisie et de choix

---
### 3 - Décorateur d'interaction et fonction d'affichage

Les fonctions de calcul utilisées dans un notebook fournissent rarement des résultats sous une forme qui permet de les connecter directement avec un interacteur. Il est donc souvent préférable de créer une **fonction d'affichage** qui va appeler la fonction de calcul et reformater les résultats obtenus pour les rendre plus lisibles. De plus, comme la fonction **`interact`** peut être utilisée sous la forme d'un **décorateur**, cela permet d'avoir une mise en oeuvre et une répartition des rôles très lisible :

- la **fonction de calcul** **`xxx(...)`** va effectuer les calculs numériques, sans prendre en compte la notion d'interaction
- le **décorateur** **`@interact(...)`** permet de créer la widget d'interaction et de définir les domaines de variation de chacun des arguments
- la **fonction d'affichage** **`show_xxx(...)`** permet de formater les résultats obtenus et de définir les valeurs par défaut pour les différents arguments de la fonction de calcul

In [ ]:
from math import factorial # mise en oeuvre avec une fonction de calcul externe

@interact(n=(0,60)) # le décorateur d'interaction définit les intervalles de variation des arguments
def show_factorial(n=0): # la fonction d'affichage définit les valeurs par défaut et formate le résultat
  print(f"{n}! = {factorial(n)}")

In [ ]:
def facto(n): # mise en oeuvre avec une fonction de calcul interne
  """return n! (factorial of integer n)"""
  out = 1
  for loop in range(1, n+1): out *= loop
  return out

@interact(n=(0,60)) # le décorateur d'interaction définit les intervalles de variation des arguments
def show_facto(n=0): # la fonction d'affichage définit les valeurs par défaut et formate le résultat
  print(f"{n}! = {facto(n)}")

---
### 4 - Interaction avec validation manuelle

Lorsque la complexité de la fonction de calcul est telle que l'affichage ne s'effectue pas de manière instantanée, ou lorsqu'il est judicieux de modifier plusieurs paramètres à la fois avant de relancer le calcul, il est possible d'utiliser le décorateur **`interact_manual`** au lieu de **`interact`** . Dans ce cas, l'affichage graphique n'est pas actualisé à chaque modification de l'un des paramètres, mais est différé jusqu'au moment où l'utilisateur clique sur le bouton intitulé ***`Run Interact`*** :

In [ ]:
from ipywidgets import interact_manual # import de la fonction 'interact_manual'

In [ ]:
@interact_manual(n=(0,1000)) # on choisit une interaction avec validation manuelle
def show_facto(n=0): # le reste ne change pas
  print(f"{n}! = {facto(n)}")

---
### 5 - Combinaison avec la fonction 'display'

Contrairement à ce que pourraient laisser penser les exemples ci-dessus, la fonction d'affichage utilisée par les interacteurs du module **`ipywidgets`** n'est pas limitée à l'utilisation de la seule fonction **`print`**. En effet, toute fonction permettant de générer un affichage au sein du notebook Jupyter est susceptible d'être employée, ce qui signifie que ces interacteurs se combinent particulièrement bien avec la fonction **`display`** étudiée dans la section B. Par exemple, la fonction **`fold_seq`** (qui génère et affiche du code HTML) ou la fonction **`chessboard`** (qui génère et affiche du code SVG) peuvent être directement pilotée par des interacteurs, avec exactement le même formalisme que les exemples précédents :

In [ ]:
print('Enter any valid Python sequence in the cell below :')
@interact_manual(fold=(1,20))
def show_fold_seq(seq='range(50)', fold=10):
  if not seq: return # si la séquence est vide, on interrompt l'exécution
  fold_seq(eval(seq), fold) # sinon, on appelle la fonction 'fold_seq' avec les paramètres interactifs

In [ ]:
@interact(size=(2,24))
def show_chessboard(size=8):
  chessboard(size) # on ne modifie qu'un seul argument de la fonction

In [ ]:
print('Enter two valid HTML colors in the cell below :')
@interact_manual(size=(2,24), scale=(1,64))
def show_chessboard(colors='#0F0 #00F', size=12, scale=24):
  if not colors: return # si les couleurs sont vide, on interrompt l'exécution
  chessboard(size, scale, colors) # sinon, on appelle la fonction 'chessboard'

<div style="padding:8px; margin:0px -20px; color:#FFF; background:#06D; text-align:right">● ● ● </div>